In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import os

# Run the setup script
%run ../config/setup_project.py

# Call the function to set up the project path
setup_project_path()

# Now you can import your modules
from src.utils import helper as h_
import code_ch_02 as f_




In [ ]:
df = pd.read_parquet('../Data/IVE_tickbidask.parq')
df

In [ ]:
# Later, you can read it back
df = pd.read_parquet('../Data/IVE_kibot.parq')
df

In [ ]:
print(df.info())

## Some Preprocessing

In [ ]:
# Some data visualization on df
# Boxplots for each column
df[['price', 'bid', 'ask', 'size', 'v', 'dv']].plot(kind='box', subplots=True, layout=(2, 3), figsize=(15, 10))
plt.show()

In [ ]:
# function to dectect, in a dataframe, rows where value is outside 
# [Q1, Q3] + 1.5 * IQR

def detect_outliers(df: pd.DataFrame, col: str='price', threshold: float=1.5):
    """Returns boolean series where True indicates the presence of an outlier"""
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return (df[col] < lower_bound) | (df[col] > upper_bound)

outliers = detect_outliers(df)

In [ ]:
df = df.loc[~outliers]
df[['price', 'bid', 'ask', 'size', 'v', 'dv']].plot(kind='box', subplots=True, layout=(2, 3), figsize=(15, 10))
plt.show()

In [ ]:
# scale columns


# Exercises

In [ ]:
"""
2.1 On a series of E-mini S&P 500 futures tick data:
(a) Form tick, volume, and dollar bars. Use the ETF trick to deal with the roll.
(b) Count the number of bars produced by tick, volume, and dollar bars on a
weekly basis. Plot a time series of that bar count. What bar type produces
the most stable weekly count? Why?
(c) Compute the serial correlation of returns for the three bar types. What bar method has the lowest serial correlation?
(d)
(e)
Partitionthebarseriesintomonthlysubsets.Computethevarianceofreturns for every subset of every bar type. Compute the variance of those variances. What method exhibits the smallest variance of variances?
Apply the Jarque-Bera normality test on returns from the three bar types. What method achieves the lowest test statistic?
"""

## 2.1 On a series of E-mini S&P 500 futures tick data:

##### (a) Form tick, volume, and dollar bars. Use the ETF trick to deal with the roll.

In [ ]:
"""2.1 On a series of E-mini S&P 500 futures tick data:
(a) Form tick, volume, and dollar bars. Use the ETF trick to deal with the roll.
"""
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# from abc import ABC, abstractmethod
# from enum import Enum

# class BarType(Enum):
#     TICK = 'tick'
#     VOLUME = 'volume'
#     DOLLAR = 'dollar'

# class BarSampling(ABC):
#     @property
#     @abstractmethod
#     def units(self):
#         """Returns:
#         {'column': col name, 'Accumulate': boolean indicating accumulation}
#         """
#         pass

#     def sample(self, df: pd.DataFrame, threshold: int = 1_000):
#         """df with columns such as 'price', 'v', 'dv'
#         Returns a list of indices of sampled bars"""
#         t = df[self.units['column']]
#         ts = 0
#         idx = []
#         for i, x in enumerate(t):
#             ts += x if self.units['Accumulate'] else 1
#             if ts >= threshold:
#                 idx.append(i)
#                 ts = 0
#         return idx

# class TickBar_Sampling(BarSampling):
#     @property
#     def units(self):
#         return {'column': 'price', 'Accumulate': False}

# class VolumeBar_Sampling(BarSampling):
#     @property
#     def units(self):
#         return {'column': 'v', 'Accumulate': True}

# class DollarBar_Sampling(BarSampling):
#     @property
#     def units(self):
#         return {'column': 'dv', 'Accumulate': True}
    
# class BarSampler:
#     def sample(self, df: pd.DataFrame, bar_type: BarType, threshold: int = 1_000):
#         """
#         df: DataFrame with columns such as 'price', 'v', 'dv'
        
#         bar_type: {'tick', 'volume', 'dollar'}
        
#         threshold: int, default 1_000
        
#         Returns a list of indices of sampled bars
        
#         """
#         if bar_type == BarType.TICK:
#             sampler = TickBar_Sampling()
#         elif bar_type == BarType.VOLUME:
#             sampler = VolumeBar_Sampling()
#         elif bar_type == BarType.DOLLAR:
#             sampler = DollarBar_Sampling()
#         else:
#             raise ValueError(f'Bar type {bar_type} not supported')
        
#         indices = sampler.sample(df, threshold)
#         return indices
    
#     def plot_bars(self, df: pd.DataFrame, threshold: int = 1_000):
#         """Plot the price series with markers for tick, volume, and dollar bars"""
#         fig, ax = plt.subplots(figsize=(14, 7))
        
#         # Plot the price series
#         ax.plot(df.index, df['price'], label='Price', color='black', alpha=0.5)
        
#         # Sample and plot Tick bars
#         tick_indices = self.sample(df, BarType.TICK, threshold=threshold)
#         ax.plot(df.index[tick_indices], df['price'].iloc[tick_indices], 'o', markersize=5, label='Tick Bars', color='blue')
        
#         # Sample and plot Volume bars
#         volume_indices = self.sample(df, BarType.VOLUME, threshold=threshold)
#         ax.plot(df.index[volume_indices], df['price'].iloc[volume_indices], 's', markersize=5, label='Volume Bars', color='red')
        
#         # Sample and plot Dollar bars
#         dollar_indices = self.sample(df, BarType.DOLLAR, threshold=threshold)
#         ax.plot(df.index[dollar_indices], df['price'].iloc[dollar_indices], '^', markersize=5, label='Dollar Bars', color='green')
        
#         # Add titles and labels
#         ax.set_title('Price Series with Tick, Volume, and Dollar Bars')
#         ax.set_xlabel('Time')
#         ax.set_ylabel('Price')
        
#         # Add a legend
#         ax.legend()
        
#         # Add a grid for better readability
#         ax.grid(True)
        
#         # Display the plot
#         plt.show()


In [ ]:
       
sampler = f_.BarSampler(df, ImbalanceBar=False)

tick_bars = sampler.sample(f_.BarType.VOLUME, thresh_mult=2.0)
len(tick_bars)

In [ ]:
TICK_THRESH = 30
MULT_THRESH = 20.0


tick_bars = sampler.sample(f_.BarType.TICK, threshold=TICK_THRESH)
volume_bars = sampler.sample(f_.BarType.VOLUME, thresh_mult=MULT_THRESH)
dollar_bars = sampler.sample(f_.BarType.DOLLAR, thresh_mult=MULT_THRESH)

price_tickBars = df['price'].iloc[tick_bars]
price_volumeBars = df['price'].iloc[volume_bars]
price_dollarBars = df['price'].iloc[dollar_bars]

# stats for each bar type (avg, std, ... )
print("Tick Bars")
print(price_tickBars.describe())
print()
print("Volume Bars")
print(price_volumeBars.describe())
print()
print("Dollar Bars")
print(price_dollarBars.describe())

In [ ]:
# plot histogram of tick, volume, and dollar bars
fig, ax = plt.subplots(3, 1, figsize=(14, 10))

sns.histplot(price_tickBars, bins=20, kde=True, color='blue', ax=ax[0])
ax[0].set_title('Tick Bars')

sns.histplot(price_volumeBars, bins=20, kde=True, color='red', ax=ax[1])
ax[1].set_title('Volume Bars')

sns.histplot(price_dollarBars, bins=20, kde=True, color='green', ax=ax[2])
ax[2].set_title('Dollar Bars')

##### (b) Count the number of bars produced by tick, volume, and dollar bars on a weekly basis. Plot a time series of that bar count. What bar type produces the most stable weekly count? Why?

In [ ]:
"""(b) Count the number of bars produced by tick, volume, and dollar bars on a 
weekly basis. Plot a time series of that bar count. What bar type produces the 
most stable weekly count? Why?
"""
print(price_volumeBars.info())

In [ ]:
tick_counts, volume_counts, dollar_counts = sampler.plot_weekly_counts(threshold=TICK_THRESH, thresh_mult=MULT_THRESH)

In [ ]:
# stats for each bar type (avg, std, ... )
print("Tick Bars")
print(tick_counts.describe())
print()
print("Volume Bars")
print(volume_counts.describe())
print()
print("Dollar Bars")
print(dollar_counts.describe())

### (c) c) Compute the serial correlation of returns for the three bar types. What bar method has the lowest serial correlation?

In [ ]:
"""c) Compute the serial correlation of returns for the three bar types. What bar
method has the lowest serial correlation?
"""

# price_tickBars, price_volumeBars, price_dollarBars

def returns(s: pd.Series):
    return np.log(s).diff().dropna()

def serial_corr(bars: pd.Series):
    return bars.autocorr()

tick_returns = returns(price_tickBars)
volume_returns = returns(price_volumeBars)
dollar_returns = returns(price_dollarBars)

print("Serial Correlation of Returns")
print("Tick Bars:", serial_corr(tick_returns))
print("Volume Bars:", serial_corr(volume_returns))
print("Dollar Bars:", serial_corr(dollar_returns))

# compared to serial correlation of all returns
all_returns = returns(df['price'])
print("Serial Correlation of All Returns:", serial_corr(all_returns))





#### d) Partition the bar series into monthly subsets. Compute the variance of returns for every subset of every bar type. Compute the variance of those variances. What method exhibits the smallest variance of variances?

In [ ]:
"""d) Partition the bar series into monthly subsets. Compute the variance of returns
for every subset of every bar type. Compute the variance of those variances.
What method exhibits the smallest variance of variances?
"""

import pandas as pd
import numpy as np

# Define a function to compute log returns
def compute_returns(price_series):
    return np.log(price_series).diff().dropna()

# Define a function to partition the series into monthly subsets
def partition_monthly(series):
    return series.groupby([series.index.month])

# Define a function to compute the variance of returns for each subset
def compute_monthly_variance(partitioned_series):
    monthly_variances = partitioned_series.apply(lambda x: compute_returns(x).var())
    return monthly_variances

# Define a function to compute the variance of variances
def compute_variance_of_variances(monthly_variances):
    return monthly_variances.var()

# Compute the variance of variances for each bar type
def variance_of_variances(price_series):
    partitioned_series = partition_monthly(price_series)
    monthly_variances = compute_monthly_variance(partitioned_series)
    variance_of_var = compute_variance_of_variances(monthly_variances)
    return variance_of_var

# Assuming price_tickBars, price_volumeBars, and price_dollarBars are defined
variance_tick = variance_of_variances(price_tickBars)
variance_volume = variance_of_variances(price_volumeBars)
variance_dollar = variance_of_variances(price_dollarBars)


print(f"Variance of Variances for Tick Bars: {variance_tick}")
print(f"Variance of Variances for Volume Bars: {variance_volume}")
print(f"Variance of Variances for Dollar Bars: {variance_dollar}")

# Determine which bar type exhibits the smallest variance of variances
bar_types = ["Tick Bars", "Volume Bars", "Dollar Bars"]
variances = [variance_tick, variance_volume, variance_dollar]

smallest_variance_of_variances = bar_types[np.argmin(variances)]
print(f"The bar type with the smallest variance of variances is: {smallest_variance_of_variances}")


### (e) Apply the Jarque-Bera normality test on returns from the three bar types. What method achieves the lowest test statistic?

In [ ]:
"""(e) Apply the Jarque-Bera normality test on returns from the three bar types.
What method achieves the lowest test statistic?
"""

import pandas as pd
import numpy as np
from scipy.stats import jarque_bera

# Define a function to compute log returns
def compute_returns(price_series):
    return np.log(price_series).diff().dropna()

# Compute the Jarque-Bera test statistic and p-value
def compute_jarque_bera(returns):
    return jarque_bera(returns)

# Assuming price_tickBars, price_volumeBars, and price_dollarBars are defined
tick_returns = compute_returns(price_tickBars)
volume_returns = compute_returns(price_volumeBars)
dollar_returns = compute_returns(price_dollarBars)

# Compute the Jarque-Bera test for each bar type
jb_tick = compute_jarque_bera(tick_returns)
jb_volume = compute_jarque_bera(volume_returns)
jb_dollar = compute_jarque_bera(dollar_returns)

print(f"Jarque-Bera Test for Tick Bars: Statistic={jb_tick.statistic}, p-value={jb_tick.pvalue}")
print(f"Jarque-Bera Test for Volume Bars: Statistic={jb_volume.statistic}, p-value={jb_volume.pvalue}")
print(f"Jarque-Bera Test for Dollar Bars: Statistic={jb_dollar.statistic}, p-value={jb_dollar.pvalue}")

# Determine which method achieves the lowest test statistic
jb_statistics = {
    "Tick Bars": jb_tick.statistic,
    "Volume Bars": jb_volume.statistic,
    "Dollar Bars": jb_dollar.statistic
}

lowest_jb = min(jb_statistics, key=jb_statistics.get)
print(f"The method with the lowest Jarque-Bera test statistic is: {lowest_jb}")


In [ ]:
price_dollarBars

## 2.2 On a series of E-mini S&P 500 futures tick data, compute dollar bars and dollar imbalance bars. What bar type exhibits greater serial correlation? Why?

In [ ]:
"""On a series of E-mini S&P 500 futures tick data, compute dollar bars
and dollar imbalance bars. What bar type exhibits greater serial correlation?
Why?
"""

In [ ]:
# Just years 2020>
df_sample = df.loc['2023':]
T0 = 50

# sampler = f_.BarSampler(df_sample, ImbalanceBar=True)

# ids = sampler.sample(f_.BarType.VOLUME, t0 = T0)
# ids

In [ ]:
imbalance_smapler = f_.BarSampler(df_sample, ImbalanceBar=True)

samples = imbalance_smapler.sample(f_.BarType.DOLLAR, t0=T0)
samples

In [ ]:
# histogram of prices at indices 'samples'

df_sample['price'].iloc[samples]

tmp = df_sample['price'].iloc[samples]
sns.histplot(tmp, bins=20, kde=True, color='blue')
plt.title('Dollar Imbalance Bars')
plt.show()


In [ ]:



results = imbalance_smapler.parallel_sample(t0=T0)

tick_ImbBars, volume_ImbBars, dollar_ImbBars = results.values()


In [ ]:
tick_ImbBars

In [ ]:

price_tickImbBars = df['price'].iloc[tick_ImbBars]
price_volumeImbBars = df['price'].iloc[volume_ImbBars]
price_dollarImbBars = df['price'].iloc[dollar_ImbBars]

# stats for each bar type (avg, std, ... )
print("Tick Imbalance Bars")
print(price_tickImbBars.describe())
print()
print("Volume Imbalance Bars")
print(price_volumeImbBars.describe())
print()
print("Dollar Imbalance Bars")
print(price_dollarImbBars.describe())

def returns(s: pd.Series):
    return np.log(s).diff().dropna()

def serial_corr(bars: pd.Series):
    return bars.autocorr()

tickImb_returns = returns(price_tickImbBars)
volumeImb_returns = returns(price_volumeImbBars)
dollarImb_returns = returns(price_dollarImbBars)

print("Serial Correlation of Returns")
print("Tick Imbalance Bars:", serial_corr(tickImb_returns))
print("Volume Imbalance Bars:", serial_corr(volumeImb_returns))
print("Dollar Imbalance Bars:", serial_corr(dollarImb_returns))

# compared to serial correlation of all returns
all_returns = returns(df['price'])
print("Serial Correlation of All Returns:", serial_corr(all_returns))



In [ ]:
"""
Moments for each (kurto, skew, ...)
tickImb_returns
volumeImb_returns
dollarImb_returns 
"""

print("Moments for each bar type")
print("Tick Imbalance Bars")
print("Kurtosis:", tickImb_returns.kurtosis())
print("Skewness:", tickImb_returns.skew())
print()
print("Volume Imbalance Bars")
print("Kurtosis:", volumeImb_returns.kurtosis())
print("Skewness:", volumeImb_returns.skew())
print()
print("Dollar Imbalance Bars")
print("Kurtosis:", dollarImb_returns.kurtosis())
print("Skewness:", dollarImb_returns.skew())




In [ ]:
tickImb_returns